In [3]:
!pip install pandas
!pip install requests



In [ ]:
import requests
import json
from datetime import datetime

# --- Configuration ---
# L'URL de base du portail API Fiware de Montpellier
BASE_URL = "https://portail-api-data.montpellier3m.fr"

# L'identifiant complet de l'entité (URN) :
ECOCOUNTER_URN = "urn:ngsi-ld:EcoCounter:X2H22104766"

# Définition de l'endpoint générique
ENDPOINT_TEMPLATE = "/ecocounter_timeseries/{ecocounterId}/attrs/intensity"

# 1. PARAMÈTRES DE REQUÊTE : Début et Fin de l'intervalle

# Date de début (ex: 1er janvier 2025)
FROM_DATE_STR = "2023-01-01T00:00:00"

# Date de fin (ex: Hier, 24 novembre 2025 à minuit, car nous sommes le 25 novembre)
# C'est une bonne pratique de s'arrêter à la veille pour les données Open Data.
TO_DATE_STR = "2023-12-31T23:59:59" 

url = BASE_URL + ENDPOINT_TEMPLATE.format(ecocounterId=ECOCOUNTER_URN)
    
# Paramètres de la requête GET
params = {
        "fromDate": FROM_DATE_STR, 
        "toDate": TO_DATE_STR, 
        "limit": 1000 # Limite de résultats
    }

# Headers de la requête
headers = {
        "Accept": "application/json",
        # Décommentez si besoin : "Authorization": f"Bearer {API_KEY}" 
    }
    
try:
    print(f"Tentative de connexion à l'URL : {url}")
    print(f"Période demandée : du {FROM_DATE_STR} au {TO_DATE_STR}")
    
    response = requests.get(url, headers=headers, params=params)

    response.raise_for_status() 
    
    data = response.json()
    
    print("\n Requête réussie ! Aperçu des données :")
    print(json.dumps(data))
     

except requests.exceptions.HTTPError as err:
    print(f"\n Erreur HTTP ({response.status_code}): {err}")
    print("Erreur : Problème d'authentification ou ID de compteur invalide.")
except requests.exceptions.RequestException as e:
    print(f"\n Erreur de connexion: {e}")

  

Tentative de connexion à l'URL : https://portail-api-data.montpellier3m.fr/ecocounter_timeseries/urn:ngsi-ld:EcoCounter:X2H22104766/attrs/intensity
Période demandée : du 2023-01-01T00:00:00 au 2023-12-31T23:59:59



 Requête réussie ! Aperçu des données :
{"attrName": "intensity", "entityId": "urn:ngsi-ld:EcoCounter:X2H22104766", "entityType": "EcoCounter", "index": ["2023-01-01T00:00:00.000+00:00", "2023-01-01T01:00:00.000+00:00", "2023-01-01T04:00:00.000+00:00", "2023-01-01T05:00:00.000+00:00", "2023-01-01T07:00:00.000+00:00", "2023-01-01T08:00:00.000+00:00", "2023-01-01T09:00:00.000+00:00", "2023-01-01T11:00:00.000+00:00", "2023-01-01T13:00:00.000+00:00", "2023-01-01T14:00:00.000+00:00", "2023-01-01T15:00:00.000+00:00", "2023-01-01T17:00:00.000+00:00", "2023-01-01T23:00:00.000+00:00", "2023-01-02T01:00:00.000+00:00", "2023-01-02T02:00:00.000+00:00", "2023-01-02T03:00:00.000+00:00", "2023-01-02T07:00:00.000+00:00", "2023-01-02T08:00:00.000+00:00", "2023-01-02T09:00:00.000+00:00", "2023-01-02T10:00:00.000+00:00", "2023-01-02T12:00:00.000+00:00", "2023-01-02T17:00:00.000+00:00", "2023-01-02T18:00:00.000+00:00", "2023-01-02T21:00:00.000+00:00", "2023-01-03T00:00:00.000+00:00", "2023-01-03T05:00:00

In [ ]:
import pandas as pd

df= pd.DataFrame(data)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attrName    1000 non-null   object 
 1   entityId    1000 non-null   object 
 2   entityType  1000 non-null   object 
 3   index       1000 non-null   object 
 4   values      1000 non-null   float64
dtypes: float64(1), object(4)
memory usage: 39.2+ KB


In [ ]:
print(df['entityId'].count())

In [ ]:
# import plotly.graph_objects as go
# import ipywidgets as widgets
# from ipywidgets import interact

# def plot_raw_zoomable(df, variable):
#     """
#     Affiche les données BRUTES (sans aucune moyenne) avec des outils de zoom temporels puissants.
#     """
#     # Création de la figure vide
#     fig = go.Figure()

#     # Ajout de la courbe (Trace) des données brutes
#     # On utilise ScatterGL au lieu de Scatter standard car c'est beaucoup plus rapide 
#     # pour afficher des milliers de points (une année = 8760 points)
#     fig.add_trace(go.Scattergl(
#         x=df.index, 
#         y=df[variable],
#         mode='lines',
#         name=variable,
#         line=dict(width=1.5, color='#1f77b4')
#     ))

#     # --- CONFIGURATION DU ZOOM ET DES BOUTONS (C'est là que la magie opère) ---
#     fig.update_layout(
#         title=f"Visualisation Brute : {variable}",
#         xaxis=dict(
#             title="Temps",
            
#             # 1. Le Range Slider (La barre de défilement en bas)
#             rangeslider=dict(visible=True),
#             type="date",
            
#             # 2. Les Boutons de Zoom Rapide (Hour, Day, Month, Year)
#             rangeselector=dict(
#                 buttons=list([
#                     dict(count=1, label="1h", step="hour", stepmode="backward"),
#                     dict(count=24, label="24h (Jour)", step="hour", stepmode="backward"),
#                     dict(count=1, label="1 Mois", step="month", stepmode="backward"),
#                     dict(count=6, label="6 Mois", step="month", stepmode="backward"),
#                     dict(count=1, label="1 An", step="year", stepmode="backward"),
#                     dict(step="all", label="Tout")
#                 ])
#             )
#         ),
#         yaxis=dict(title=variable),
#         template="plotly_white",
#         height=600 # On fait un graphique assez haut pour bien voir
#     )

#     fig.show()

# # --- Lancement Interactif ---
# cols_dispo = ['temperature_2m', 'wind_speed_10m', 'rain', 'precipitation']

# print("🔍 --- EXPLORATEUR DE DONNÉES BRUTES ---")
# print("Utilise les boutons '1h', '24h', '1 Mois' en haut du graphique pour changer l'échelle.")

# interact(plot_raw_zoomable, 
#          df=widgets.fixed(df_clean),
#          variable=widgets.Dropdown(options=cols_dispo, value='temperature_2m', description='Donnée :')
# );

🔍 --- EXPLORATEUR DE DONNÉES BRUTES ---
Utilise les boutons '1h', '24h', '1 Mois' en haut du graphique pour changer l'échelle.


NameError: name 'df_clean' is not defined